# 🎬 UVG MAX - Premium Video Generator

**Automated Video Generation with TTS**

---

In [ ]:
#@title 🚀 Cell 1: Setup Everything

import os, sys

print('📦 Cloning repository...')
os.chdir('/content')
!rm -rf uvg-max-engine
!git clone https://github.com/Shya-Bubu/uvg-max-engine.git
os.chdir('uvg-max-engine')
sys.path.insert(0, '/content/uvg-max-engine')

print('\n📦 Installing dependencies...')
%pip install -q python-dotenv requests tqdm Pillow numpy
%pip install -q gTTS

print('\n🔑 Setting API keys...')
os.environ['PEXELS_KEY'] = '7QAyreSxu44EAJYIRfYXstHpvj1tof7v3Baj8tv8fvO4DV6l16I2FzlY'
os.environ['PIXABAY_KEY'] = '53451868-98a1100fbad21a5918d9610dd'

print('\n✅ SETUP COMPLETE!')

In [ ]:
#@title 🧪 Cell 2: Test TTS

import sys
if '/content/uvg-max-engine' not in sys.path:
    sys.path.insert(0, '/content/uvg-max-engine')

from uvg_core.edge_tts_adapter import EdgeTTSAdapter

print('🎙️ Testing TTS with gTTS...')
tts = EdgeTTSAdapter()
result = tts.synthesize(
    text='Hello! This is a test of the text to speech system.',
    output_name='test_voice'
)

if result.success:
    print(f'✅ Audio generated: {result.audio_path}')
    from IPython.display import Audio
    display(Audio(result.audio_path))
else:
    print(f'❌ Failed: {result.error}')

In [ ]:
#@title 🎬 Cell 3: Generate Video

import sys, os
if '/content/uvg-max-engine' not in sys.path:
    sys.path.insert(0, '/content/uvg-max-engine')
os.chdir('/content/uvg-max-engine')

SCRIPT = {
    "version": "2.1",
    "video_meta": {
        "title": "The Power of Focus",
        "orientation": "portrait",
        "resolution": {"width": 1080, "height": 1920}
    },
    "scenes": [
        {"scene_id": 1, "text": "In a world of distractions, focus is your superpower.", "search_keywords": "meditation peaceful"},
        {"scene_id": 2, "text": "Every great achievement started with a moment of clarity.", "search_keywords": "mountain sunrise"},
        {"scene_id": 3, "text": "Small steps every day lead to massive results.", "search_keywords": "running fitness"},
        {"scene_id": 4, "text": "Today, choose focus. Tomorrow, celebrate success.", "search_keywords": "celebration happy"}
    ]
}

from uvg_core.uvg_pipeline import run_from_json

print('🎬 Starting video generation...')
result = run_from_json(SCRIPT)

if result.success:
    print(f'\n✅ Video: {result.output_path}')
else:
    print('\n❌ Failed:', result.errors)

In [ ]:
#@title 📺 Cell 4: Play Video

from IPython.display import Video
import glob, os
os.chdir('/content/uvg-max-engine')

videos = glob.glob('uvg_output/**/*.mp4', recursive=True)
if videos:
    latest = max(videos, key=os.path.getctime)
    print(f'🎬 {latest}')
    display(Video(latest, embed=True, width=400))
else:
    print('❌ No video. Run Cell 3 first.')

In [ ]:
#@title 📥 Cell 5: Download

from google.colab import files
import glob, os
os.chdir('/content/uvg-max-engine')

videos = glob.glob('uvg_output/final/*.mp4')
if videos:
    files.download(max(videos, key=os.path.getctime))
else:
    print('❌ No video.')